
## To induce SP in rates

The Berekely admissions version of SP is what we will call rate based.  This requires 


2 grouping varibles & a binary
 - binary `decision` variable that will have a rate that flips (eg admission decision)
 - 1 `protected class` grouping variable that that will be how the rates are compared, eg gender (likely 2 levels, can possibly generalize)
 - 1 `explanatory` grouping variable eg has class imbalance
 
An open problem is to determine if we can define a mixed case where we use continuous variables to define the decsion variable.




In [1]:
import numpy as np
import pandas as pd
import string
import matplotlib.pylab as plt

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


The causal explanation for rate-based SP is that the application rates by gender varied in a way that was correlated with the department size and/or acceptance rate.  Here we create a model that will, for most samples, have SP for some departments. We only set the portion of applications to each department, the rate of each gender applying to each department and each department's acceptance rate.  

In [2]:
p_dept = [.15,.15,.1,.6]
p_gender_dept = [[.7, .3],[.8,.2],[.85,.15],[.2,.8]]
gender_list = ['F','M']
p_admit_dept = [[.15,.85], [.18,.82], [.25,.75],[.3,.7]]
# need to have higher accept in the larger subgroup, larger subgroup should have opposite protected class balance
N = 1000

d = np.random.choice(list(range(len(p_dept))), size=N, p =p_dept)
g = [np.random.choice(gender_list, p =p_gender_dept[d_i]) for d_i in d]
a = [np.random.choice([1,0],p = p_admit_dept[d_i]) for d_i in d]
data = [[d_i,g_i,a_i] for d_i, g_i,a_i in zip(d,g,a)]

df = pd.DataFrame(data = data, columns=['department','gender','decision'])

p_race_dept = [[.8,.1,.1], [.7,.13,.17],[.5,.2,.3],[.85,.07,.08]]
race_list = ['W','B','H']

r = [np.random.choice(race_list, p =p_race_dept[d_i]) for d_i in d]
df['race'] = r
df.head()

,department,gender,decision,race
0,0,F,1,H
1,0,M,0,W
2,1,F,0,W
3,3,M,0,W
4,1,F,0,W


We can check that the probabilities match what we set.  First the per-department admission rate.

In [3]:
actual_admit_dept = df.groupby(['department']).mean()
expected_admit_dept = [p[0] for p in p_admit_dept]
actual_admit_dept['expected'] = expected_admit_dept
actual_admit_dept

,decision,expected
department,,
0,0.090323,0.15
1,0.253247,0.18
2,0.314815,0.25
3,0.276158,0.30


In [4]:
actual_app_dept = df.groupby(['department'])['gender'].value_counts().unstack()
actual_app_dept

gender,F,M
department,,
0,113,42
1,115,39
2,94,14
3,125,458


In [5]:
expected_app_dept_dat = [[p_d[0]*n_d, p_d[1]*n_d] for p_d,n_d in zip(p_gender_dept,[p*N for p in p_dept])]
expected_app_dept = pd.DataFrame(data = expected_app_dept_dat, columns = gender_list)
expected_app_dept

,F,M
0,105.0,45.0
1,120.0,30.0
2,85.0,15.0
3,120.0,480.0


Next we can look for SP

In [6]:
df.groupby('gender')['decision'].mean()

gender
F    0.234899
M    0.258590
Name: decision, dtype: float64

In [7]:
df.groupby(['gender','department']).mean().unstack()

decision                              
department         0         1         2         3
gender                                            
F           0.079646  0.234783  0.319149  0.312000
M           0.119048  0.307692  0.285714  0.266376

To detect SP, we compare which group is the highest in the overall to to which group is highest in each of the departments. 

In [8]:
overall_dat = df.groupby('gender')['decision'].mean()
per_dept_dat = df.groupby(['gender','department']).mean().unstack()

overall_dat.values[0]/overall_dat.values[1]
per_dept_dat.values[0]/per_dept_dat.values[1] /(overall_dat.values[0]/overall_dat.values[1])

array([0.73649954, 0.83999832, 1.22967566, 1.28940506])

In [9]:
per_dept_dat.values[0]/per_dept_dat.values[1] /(overall_dat.values[0]/overall_dat.values[1])

array([0.73649954, 0.83999832, 1.22967566, 1.28940506])

In [10]:
df.to_csv('synthetic_admission_high_sp.csv',index=False)

In [11]:
# now detect and compare which trends
overall = df.groupby('gender')['decision'].mean().idxmax()
print('overall more admitted: ', overall)

per_dept = [gender_list[g] for g in np.argmax(df.groupby(['gender','department']).mean().unstack().values,axis=0)]
print('per dept more addmitted:', per_dept)
# it's sp for each dept that is not the same as the overall
[not(dept==overall) for dept in per_dept]

overall more admitted:  M
per dept more addmitted: ['M', 'M', 'F', 'F']


[False, False, True, True]

Now, we can run it a bunch more times and count how many SP in each trial, to see how reliable it is.

In [12]:
sp_occ = []

for i in range(20):
    p_dept = [.15,.15,.1,.6]
    p_gender_dept = [[.7, .3],[.8,.2],[.85,.15],[.2,.8]]
    gender_list = ['F','M']
    p_admit_dept = [[.15,.85], [.18,.82], [.25,.75],[.3,.7]]
    # need to have higher accept in the larger subgroup, larger subgroup should have opposite protected class balance
    N = 1000

    d = np.random.choice(list(range(len(p_dept))), size=N, p =p_dept)
    g = [np.random.choice(gender_list, p =p_gender_dept[d_i]) for d_i in d]
    a = [np.random.choice([1,0],p = p_admit_dept[d_i]) for d_i in d]
    data = [[d_i,g_i,a_i] for d_i, g_i,a_i in zip(d,g,a)]

    df = pd.DataFrame(data = data, columns=['department','gender','decision'])

    overall = df.groupby('gender')['decision'].mean().idxmax()
    per_dept = [gender_list[g] for g in np.argmax(df.groupby(['gender','department']).mean().unstack().values,axis=0)]


    sp_occ.append(sum([not(dept==overall) for dept in per_dept]))
    
sp_occ

[2, 4, 3, 2, 2, 3, 2, 2, 1, 3, 4, 3, 3, 2, 0, 2, 1, 0, 2, 2]

We see from above that it generates SP most of the time but a varialbe number of times for the exact same settings to this sampling method does not reliably induce SP, it does show that, with no malintent that SP can occur.  

# intentional SP

To more reliably induce rate based SP we should sample less independently.  


In [49]:
p_dept = [.15,.2,.1,.55]
p_gender_dept = [[.7, .3],[.8,.2],[.85,.15],[.2,.8]]
gender_list = ['F','M']
p_admit_dept_gender = [{'F':.18,'M':.12},{'F':.17,'M':.1},
                       {'F':.30,'M':.27},{'F':.35,'M':.30}] 
# need to have higher accept in the larger subgroup, larger subgroup should have opposite protected class balance
N = 1000

d = np.random.choice(list(range(len(p_dept))), size=N, p =p_dept)
g = [np.random.choice(gender_list, p =p_gender_dept[d_i]) for d_i in d]
p_admit =[ p_admit_dept_gender[d_i][g_i] for d_i,g_i in zip(d,g)]
p_admit
a = [np.random.choice([1,0], p = [p,1-p]) for p in p_admit]
data = [[d_i,g_i,a_i] for d_i, g_i,a_i in zip(d,g,a)]

df = pd.DataFrame(data = data, columns=['department','gender','decision'])
df.head()

,department,gender,decision
0,1,F,0
1,0,F,0
2,1,F,0
3,3,F,1
4,0,F,0


In [50]:
p_admit_dept_gender[1]['M']

0.1

In [51]:
df.groupby('gender')['decision'].mean()

gender
F    0.208716
M    0.265957
Name: decision, dtype: float64

In [52]:
df.groupby(['gender','department']).mean().unstack()

decision                              
department         0         1         2         3
gender                                            
F           0.168539  0.111842  0.337349  0.276786
M           0.140000  0.125000  0.200000  0.295154

In [53]:
overall_dat = df.groupby('gender')['decision'].mean()
per_dept_dat = df.groupby(['gender','department']).mean().unstack()

overall_dat.values[0]/overall_dat.values[1]
per_dept_dat.values[0]/per_dept_dat.values[1] /(overall_dat.values[0]/overall_dat.values[1])

array([1.53401805, 1.14012527, 2.14935027, 1.19495617])

In [54]:
overall = df.groupby('gender')['decision'].mean().idxmax()

per_dept = [gender_list[g] for g in np.argmax(df.groupby(['gender','department']).mean().unstack().values,axis=0)]


[not(dept==overall) for dept in per_dept]

[True, False, True, False]

We can try to use 3 categories and do an ordering swap detection, maybe?

In [24]:

p_race_dept = [[.8,.1,.1], [.7,.13,.17],[.5,.2,.3],[.85,.07,.08]]
race_list = ['W','B','H']

r = [np.random.choice(race_list, p =p_race_dept[d_i]) for d_i in d]
df['race'] = r

In [25]:
df.groupby('race')['decision'].mean()

race
B    0.292453
H    0.228346
W    0.289439
Name: decision, dtype: float64

In [26]:
df.groupby(['race','department']).mean().unstack()

decision                              
department         0         1         2         3
race                                              
B           0.187500  0.185185  0.440000  0.315789
H           0.333333  0.128205  0.258065  0.261905
W           0.152542  0.153846  0.285714  0.355102

In [18]:
df.head()

,department,gender,decision
0,2,F,0
1,3,M,0
2,0,F,0
3,0,F,0
4,3,M,0


# Generalizing this framework

First we provide general variables and separate the set parts from the sampling. 

In [55]:
# must have imbalance
p_explanatory = [.15,.2,.1,.55]
#protected, given explantory, largest explantory should have fipped rates
# larger subgroup should have opposite protected class balance
p_protected_explanatory = [[.7, .3],[.8,.2],[.85,.15],[.2,.8]]

# need to have higher accept in the larger subgroup,
p_outcome_all = [{'F':.18,'M':.12},{'F':.17,'M':.1},
                       {'F':.30,'M':.27},{'F':.35,'M':.30}]
N = 1000

def gen_rate(N, p_explanatory, p_protected_explanatory,p_outcome_all):
    """
    sampler that takes in probabilities 
    
    Parameters
    ----------
    N : scalar
        number of samples to draw
    """
    protected_list = ['F','M']
    explantory = np.random.choice(list(range(len(p_explanatory))),
                                    size=N, p =p_dept)
    protected = [np.random.choice(protected_list, p=p_protected_explanatory[e])
                                    for e in explantory]
    p_outcome =[ p_outcome_all[e][p] for e,p in zip(explantory,protected)]

    outcome = [np.random.choice([1,0], p = [p,1-p]) for p in p_outcome]
    data = [[e,p,o] for e,p,o in zip(explantory,protected,outcome)]

    df = pd.DataFrame(data = data, columns=['explanatory','protected','outcome'])
    
    return df

In [56]:
df_gen = gen_rate(N, p_explanatory, p_protected_explanatory,p_outcome_all)
df_gen.head()

,explanatory,protected,outcome
0,3,M,0
1,3,M,0
2,1,F,0
3,3,F,0
4,3,M,1


In [57]:
df_gen = gen_rate(N, p_explanatory, p_protected_explanatory,p_outcome_all)
df_gen.head()

,explanatory,protected,outcome
0,3,F,1
1,3,M,0
2,3,M,0
3,3,M,1
4,2,F,1


Next, we need to determin how to sample the probabilities such that we get the balances needed and how to parameterize. To facilitate this, we'll make a sample detector for testing. 

In [58]:

def check_sp_rate(df_rate):
    """rate specific sp detector"""
    overall = df_rate.groupby('protected')['outcome'].mean().idxmax()

    per_group = df_rate.groupby(['protected','explanatory']).mean().unstack().idxmax()

    
    return [not(group==overall) for group in per_group]


In [59]:
check_sp_rate(df_gen)

[True, False, True, True]

In [60]:
sum(check_sp_rate(df_gen))

3

In [62]:
sp_counts = []
for i in range(100):
    sp_counts.append(sum(check_sp_rate(gen_rate(N, p_explanatory, p_protected_explanatory,p_outcome_all))))

sp_counts

[3,
 3,
 2,
 1,
 4,
 2,
 2,
 3,
 4,
 2,
 2,
 4,
 4,
 3,
 1,
 0,
 4,
 4,
 0,
 3,
 3,
 4,
 3,
 1,
 4,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 1,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 2,
 4,
 1,
 3,
 3,
 2,
 4,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 2,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 0,
 0,
 2,
 3,
 3,
 3,
 2,
 4,
 3,
 3,
 2,
 2,
 3,
 3,
 3,
 1,
 4,
 4,
 3,
 3,
 0,
 3,
 3,
 2,
 4,
 0,
 4,
 3,
 4,
 2,
 3,
 3,
 2,
 4,
 4,
 4,
 3]

In [66]:
df_gen.groupby('explanatory').groupby('protected').describe()

AttributeError: Cannot access callable attribute 'groupby' of 'DataFrameGroupBy' objects, try using the 'apply' method